In [1]:
from src import preprocess
reload(preprocess)
from src import graph
reload(graph)
from src import file_io
reload(file_io)
from src import publish
reload(publish)

import os
import urlparse
import logging
from collections import defaultdict
import joblib as jl
import pandas as pd

from src.graph import build_graph, build_graph_mp
from src.preprocess import make_domain_path_split, ac_map_local
from src.file_io import load_urls, dump_urls, dump_edge_list, dump_node_list, load_label_data
from src.publish import illegal_domain_evaluate,  illegal_domain_evaluate_mp
from src.logger import logger
logger.setLevel(logging.INFO)

In [2]:
work_dir = "./data/EXP_1"

In [3]:
ac_white_domain = file_io.load_ac_domain(ac_root_path = "../data/ac_white")

ac_gambing_domain = load_urls("../data/ac_white/赌博", csv = False)
ac_porn_domain = load_urls("../data/ac_white/色情", csv = False)
ac_gov_domain = load_urls("../data/ac_white/政府机构", csv = False)
ac_edu_domain = load_urls("../data/ac_white/教育", csv = False)

cmt_porn, cmt_gambing, cmt_others, cmt_die = load_label_data("../data")

INFO:url_network:ac white domain list load:9161480
INFO:url_network:
porn:557
gambing:64
die:34
others:163


In [4]:
known_gambing = set(cmt_gambing) | set(ac_gambing_domain)
known_porn = set(cmt_porn) | set(ac_porn_domain)

In [5]:
df = pd.read_csv("../data/sangfor/safe_0419.csv")
urls = list(df.url)

# evaluate porn

In [94]:
trainset_domain = set(ac_porn_domain)
regex_porn = file_io.load_regex_list(os.path.join(work_dir,"porn_regex_raw.txt"))

In [48]:
df, fp_map, detect_map = illegal_domain_evaluate_mp(urls,  
                                     regex_porn,
                                     ac_white_domain,
                                     trainset_domain, 
                                     result_file_path = os.path.join(work_dir,"porn_regex_res.txt"),
                                     n_jobs = 8)

INFO:url_network:preprocess time consume:11.837890


                       /template/115porn	independent detection 0	tp 3	fp 0
                         /css/kedou\.css	independent detection 10	tp 3	fp 0
                /images/search_icon\.png	independent detection 0	tp 3	fp 0
                    /timev/ad_mode\.html	independent detection 38	tp 5	fp 0
                          /im20/css\.css	independent detection 1	tp 3	fp 0
                             /page/\d{1}	independent detection 366	tp 6	fp 23
               /portal\.php/favicon\.ico	independent detection 0	tp 2	fp 0
    /Scripts/jquery-ui-1\.8\.20\.min\.js	independent detection 6	tp 4	fp 1
                          /content/\d{3}	independent detection 811	tp 78	fp 26
                       /packs/vod_player	independent detection 20	tp 7	fp 1
                      /includes/url\.php	independent detection 1	tp 4	fp 0
     /packs/player/ckplayer/ckplayer\.js	independent detection 2	tp 2	fp 0
                         /adimg/555\.gif	independent detection 0	tp 2	fp 0
               

               /template/qu_115style/img	independent detection 18	tp 4	fp 0
                         /images/pc\.css	independent detection 17	tp 2	fp 0
/video1/jxfikigetqwalfopxbrmuvrjrvtwvc\.mp4	independent detection 1	tp 4	fp 0
             /other/moicon_2af3d33\.woff	independent detection 0	tp 2	fp 0
    /iframeplayer/videojs/video\.min\.js	independent detection 15	tp 2	fp 0
                        /images/250\.gif	independent detection 1	tp 2	fp 0
                                 /inxy/m	independent detection 1	tp 90	fp 0
                  /ckplayX/ckplayer\.swf	independent detection 2	tp 3	fp 0
/iframeplayer/videojs/video-js\.min\.css	independent detection 16	tp 3	fp 0
                      /ftt2/toplists/img	independent detection 1	tp 3	fp 0
                   /avoinx/content/\d{3}	independent detection 11	tp 30	fp 0
          /Tpl/xxxx/images/av_style\.css	independent detection 1	tp 2	fp 0
                            /app/go\.asp	independent detection 5	tp 6	fp 0
               

In [50]:
df = publish.regex_publish(input_path = os.path.join(work_dir,"porn_regex_res.txt"),
                          output_path = os.path.join(work_dir,"porn_regex_publish.txt"),
                          publish_score = 1,
                          publish_fp_thresh=1,
                          publish_tp_thresh=1)

INFO:url_network:regular expression publish	83


In [98]:
predict_domain, predict_dict = publish.make_prediction(urls, 
                                                       input_path= os.path.join(work_dir,"porn_regex_publish.txt"), 
                                                       n_jobs=8,
                                                       verbose = True)
jl.dump([predict_domain, predict_dict], "./data/predict_porn.jl.z")

batch index 0	sample index 0	hit url 1	/images/pc\.css
batch index 1	sample index 0	hit url 1	/img/tu\.css
batch index 0	sample index 1	hit url 1	/images/bg\.png
batch index 1	sample index 1	hit url 1	/static/main\.css
batch index 2	sample index 0	hit url 2	/g/logo
batch index 0	sample index 2	hit url 4	/ftt2/toplists/img
batch index 1	sample index 2	hit url 91	/inxy/m/\d{3}
batch index 2	sample index 1	hit url 1	/sj/Tpl/baibaiwap//style/style\.css
batch index 3	sample index 0	hit url 2	/Tpl/custom//style/images
batch index 0	sample index 3	hit url 1	/msvod/tpl/skins/msvod/js/jquery\.SuperSlide\.2\.1\.1\.js
batch index 2	sample index 2	hit url 1	/ads/dh\.js
batch index 4	sample index 0	hit url 1	/packs/player/ckplayer/ckplayer\.js
batch index 3	sample index 1	hit url 5	/template/menghuan
batch index 1	sample index 3	hit url 1	/images/tpl_input_bg\.gif
batch index 0	sample index 4	hit url 5	/packs/vod_player
batch index 4	sample index 1	hit url 1	/url/p\.js
batch index 2	sample index 3	

In [107]:
predict_domain, predict_dict =  jl.load("./data/predict_porn.jl.z")
predict_domain = set(predict_domain) - set(cmt_die) - set([_ for k, __ in fp_map.iteritems() for _ in __])
new_detect = predict_domain - known_porn

print "total detection:%d" %len(predict_domain)
print "predict porn(trainset/independent):\t%d/%d" %(len(predict_domain & set(ac_porn_domain)), len(predict_domain & set(cmt_porn)))
print "new detect:\t%d" %len(new_detect)

dump_urls(list(new_detect), "./data/predict_porn.txt")

total detection:723
predict porn(trainset/independent):	222/46
new detect:	455


# evaluate gambing

In [67]:
trainset_domain = set(ac_gambing_domain)
regex_gambing = file_io.load_regex_list(os.path.join(work_dir,"gambing_regex_raw.txt"))

In [68]:
df, fp_map, detect_map = illegal_domain_evaluate_mp(urls,  
                                     regex_gambing,
                                     ac_white_domain,
                                     trainset_domain, 
                                     result_file_path = os.path.join(work_dir,"gambing_regex_res.txt"),
                                     n_jobs = 8)

INFO:url_network:preprocess time consume:12.055115


                         /images/ta\.gif	independent detection 0	tp 2	fp 0
                      /images/mark1\.gif	independent detection 1	tp 2	fp 0
                       /api/download\.do	independent detection 27	tp 67	fp 2
                        /gg/3300518\.gif	independent detection 0	tp 2	fp 0
                            /tu/cbw\.gif	independent detection 1	tp 2	fp 0
                           /css/99z\.css	independent detection 0	tp 2	fp 0
                              /gg/5\.htm	independent detection 0	tp 2	fp 0
                       /789_files/3\.gif	independent detection 0	tp 2	fp 0
                       /manhua/mhxj\.jpg	independent detection 1	tp 2	fp 0
                            /tu/mhb\.jpg	independent detection 0	tp 2	fp 0
                       /tuku/tu/v05\.jpg	independent detection 0	tp 2	fp 0
                             /tu/gp\.jpg	independent detection 1	tp 2	fp 0
     /Index/Public/JavaScript/bmjs00\.js	independent detection 6	tp 2	fp 1
                       

                     /bbs/img/zzyy1\.jpg	independent detection 0	tp 2	fp 0
                         /dltu/gif1\.gif	independent detection 0	tp 2	fp 0
                           /tu/01234\.js	independent detection 1	tp 2	fp 0
                            /tu/c35\.gif	independent detection 1	tp 2	fp 0
                           /js/slide\.js	independent detection 5	tp 3	fp 1
                             /tu/wx\.gif	independent detection 0	tp 2	fp 0
                                /new/img	independent detection 6	tp 2	fp 0
                            /tu/sbx\.jpg	independent detection 2	tp 2	fp 0
                        /tu/7580666\.gif	independent detection 0	tp 2	fp 0
                         /tu/kbcptu\.gif	independent detection 0	tp 2	fp 0
                     /Source/dm/d10\.htm	independent detection 1	tp 3	fp 0
                          /tu/hot_1\.gif	independent detection 1	tp 2	fp 0
                           /tu/wxdt\.gif	independent detection 5	tp 4	fp 1
                         

In [120]:
df = publish.regex_publish(input_path = os.path.join(work_dir,"gambing_regex_res.txt"),
                          output_path = os.path.join(work_dir,"gambing_regex_publish.txt"),
                          publish_score = 1,
                          publish_fp_thresh=1,
                          publish_tp_thresh=1)

INFO:url_network:regular expression publish	113


In [121]:
predict_domain, predict_dict = publish.make_prediction(urls, 
                                                       input_path= os.path.join(work_dir,"gambing_regex_publish.txt"), 
                                                       n_jobs=8,
                                                       verbose = True)
jl.dump([predict_domain, predict_dict], "./data/predict_gambing.jl.z")

batch index 0	sample index 0	hit url 1	/tu/zhong\.gif
batch index 0	sample index 1	hit url 1	/images/arrow_red\.gif
batch index 1	sample index 0	hit url 1	/images/gl\.gif
batch index 2	sample index 0	hit url 1	/tu/%E9%A6%99%E6%B8%AF%E6%8C%82%E7%89%8C\.jpg
batch index 0	sample index 2	hit url 1	/index\.files/jgflt\.gif
batch index 1	sample index 1	hit url 1	/images/lan1\.gif
batch index 3	sample index 0	hit url 1	/tu/gpxj\.jpg
batch index 2	sample index 1	hit url 1	/img-gg/9733\.gif
batch index 0	sample index 3	hit url 1	/tukuk/xpg\.jpg
batch index 1	sample index 2	hit url 1	/bo360/chat
batch index 4	sample index 0	hit url 1	/tu/ktzx\.jpg
batch index 3	sample index 1	hit url 1	/images/zhong\.gif
batch index 2	sample index 2	hit url 1	/images/bg\.gif
batch index 0	sample index 4	hit url 1	/images/d\.gif
batch index 1	sample index 3	hit url 8	/Xml/Article
batch index 4	sample index 1	hit url 1	/images/z\.gif
batch index 3	sample index 2	hit url 1	/js/dl\.js
batch index 2	sample index 3	hi

['./data/predict_gambing.jl.z']

In [166]:
predict_domain, predict_dict =  jl.load("./data/predict_gambing.jl.z")
predict_domain = set(predict_domain) - set(cmt_die) - set([_ for k, __ in fp_map.iteritems() for _ in __])
new_detect = predict_domain - known_gambing

print "total detection:%d" %len(predict_domain)
print "predict porn(trainset/independent):\t%d/%d" %(len(predict_domain & set(ac_gambing_domain)), len(predict_domain & set(cmt_gambing)))
print "new detect:\t%d" %len(new_detect)

dump_urls(list(new_detect), "./data/predict_gambing.txt")

total detection:634
predict porn(trainset/independent):	288/15
new detect:	331


# evaluate gov

In [128]:
trainset_domain = set(ac_gov_domain)
regex_gov = file_io.load_regex_list(os.path.join(work_dir,"gov_regex_raw.txt"))

In [129]:
df, fp_map, detect_map = illegal_domain_evaluate_mp(urls,  
                                     regex_gov,
                                     ac_white_domain,
                                     trainset_domain, 
                                     result_file_path = os.path.join(work_dir,"gov_regex_res.txt"),
                                     n_jobs = 8)

INFO:url_network:preprocess time consume:12.346192


         /images/jquery-1\.7\.2\.min\.js	independent detection 0	tp 2	fp 0
  /govdiropen/jcms_files/jcms1/web1/site	independent detection 0	tp 2	fp 0
                       /baweb/show/shiju	independent detection 0	tp 2	fp 0
                               /cms/html	independent detection 44	tp 35	fp 2
                            /zwgk/system	independent detection 0	tp 3	fp 0
                              /doc/\d{4}	independent detection 42	tp 25	fp 3
                /openness/detail/content	independent detection 10	tp 9	fp 0
                     /supervision/detail	independent detection 0	tp 3	fp 0
/sitefiles/bairong/scripts/independent/tracker\.js	independent detection 4	tp 2	fp 0
                              /xxgk/tzgg	independent detection 2	tp 9	fp 0
/javax\.faces\.resource/primefaces\.js\.jspx	independent detection 3	tp 3	fp 3
 /WXA11Y/soundmanager2-nodebug-jsmin\.js	independent detection 2	tp 2	fp 0
                              /ztzl/zxzt	independent detection 0	tp 6	fp 0
      

In [144]:
df = publish.regex_publish(input_path = os.path.join(work_dir,"gov_regex_res.txt"),
                          output_path = os.path.join(work_dir,"gov_regex_publish.txt"),
                          publish_score = 1,
                          publish_fp_thresh=1,
                          publish_tp_thresh=1)

INFO:url_network:regular expression publish	30


In [131]:
predict_domain, predict_dict = publish.make_prediction(urls, 
                                                       input_path= os.path.join(work_dir,"gov_regex_publish.txt"), 
                                                       n_jobs=8,
                                                       verbose = True)
jl.dump([predict_domain, predict_dict], "./data/predict_gov.jl.z")

batch index 0	sample index 0	hit url 1	/SiteFiles/bairong/scripts/independent/tracker\.js
batch index 1	sample index 0	hit url 28	/openness/detail
batch index 0	sample index 1	hit url 1	/flash/top\.swf
batch index 2	sample index 0	hit url 12	/a/\d{4}/\d{2}/\d{2}
batch index 1	sample index 1	hit url 4	/zwbz/zwdt/bmdt
batch index 0	sample index 2	hit url 3	/xhtml/js
batch index 1	sample index 2	hit url 31	/tzgg/\d{6}
batch index 2	sample index 1	hit url 11	/xxgk/tzgg
batch index 3	sample index 0	hit url 3	/styles/\d{1}/default/images
batch index 2	sample index 2	hit url 1	/flash/-1/slideshow\.swf
batch index 3	sample index 1	hit url 5	/home/content
batch index 4	sample index 0	hit url 1	/info/iList\.jsp
batch index 3	sample index 2	hit url 6	/theme/images
batch index 4	sample index 1	hit url 38	/ecdomain/framework
batch index 5	sample index 0	hit url 1	/sitefiles/assets/tracker/script\.js
batch index 4	sample index 2	hit url 42	/cms/cms
batch index 5	sample index 1	hit url 6	/atm/\d{1}
b

['./data/predict_gov.jl.z']

In [170]:
cmt_gov = []
predict_domain, predict_dict =  jl.load("./data/EXP_1/predict_gov.jl.z")
predict_domain = set(predict_domain) - set(cmt_die) - set([_ for k, __ in fp_map.iteritems() for _ in __])
new_detect = predict_domain - set(ac_gov_domain)

print "total detection:%d" %len(predict_domain)
print "predict porn(trainset/independent):\t%d/%d" %(len(predict_domain & set(ac_gov_domain)), len(predict_domain & set(cmt_gov)))
print "new detect:\t%d" %len(new_detect)

dump_urls(list(new_detect), "./data/predict_gov.txt")

total detection:160
predict porn(trainset/independent):	72/0
new detect:	88


In [172]:
set(predict_dict["/theme/images"]) & set(ac_gov_domain)

{'66pp008.com', 'www.whrsj.gov.cn'}

# evaluate edu

In [150]:
trainset_domain = set(ac_edu_domain)
regex_edu = file_io.load_regex_list(os.path.join(work_dir,"edu_regex_raw.txt"))

In [153]:
ac_white_domain_edu = set(ac_white_domain) - trainset_domain
df, fp_map, detect_map = illegal_domain_evaluate_mp(urls,  
                                     regex_edu,
                                     ac_white_domain_edu,
                                     trainset_domain, 
                                     result_file_path = os.path.join(work_dir,"edu_regex_res.txt"),
                                     n_jobs = 8)

INFO:url_network:preprocess time consume:11.616635


              /api/client/verify_channel	independent detection 9	tp 4	fp 0
                             /m/templets	independent detection 6	tp 2	fp 2
                             /content/js	independent detection 35	tp 2	fp 1
                            /wap/toutiao	independent detection 3	tp 4	fp 0
                       /js/analytics\.js	independent detection 9	tp 2	fp 0
                /tuiguang/projects/jijin	independent detection 3	tp 4	fp 0
          /newpower/baidu/search/psy/aci	independent detection 7	tp 5	fp 0
            /VideoPlay/updateStudyStatue	independent detection 0	tp 3	fp 0
                            /\d{3}/BJ/pc	independent detection 1	tp 8	fp 0
                 /templets/wodede/mobile	independent detection 0	tp 2	fp 0
                          /Public/Upload	independent detection 307	tp 2	fp 28
                                 /qg/jjs	independent detection 5	tp 6	fp 1
                        /xl/360wm/ldx/pc	independent detection 0	tp 4	fp 0
                /jife

                        /a/zhuanyeshezhi	independent detection 0	tp 4	fp 0
                 /kns/brief/result\.aspx	independent detection 5	tp 3	fp 0
                         /xl/gd/gz/360wm	independent detection 0	tp 3	fp 0
            /project/huaxijs/index\.html	independent detection 0	tp 2	fp 0
                             /renhezj/gd	independent detection 0	tp 2	fp 0
                         /jifen/shenzhen	independent detection 1	tp 8	fp 0
                         /templets/style	independent detection 12	tp 2	fp 3
          /static-dist/app/js/common\.js	independent detection 1	tp 3	fp 0
                          /skins/wap/css	independent detection 1	tp 2	fp 1
                         /xueli/shenzhen	independent detection 0	tp 6	fp 0
                  /a/zhuanyeshezhi/\d{4}	independent detection 0	tp 3	fp 0
                          /Account/Login	independent detection 8	tp 3	fp 1
                            /css/asd\.js	independent detection 0	tp 2	fp 0
           /shenzhen/tea

In [162]:
df = publish.regex_publish(input_path = os.path.join(work_dir,"edu_regex_res.txt"),
                          output_path = os.path.join(work_dir,"edu_regex_publish.txt"),
                          publish_score = 1,
                          publish_fp_thresh=0,
                          publish_tp_thresh=1)

INFO:url_network:regular expression publish	40


In [155]:
predict_domain, predict_dict = publish.make_prediction(urls, 
                                                       input_path= os.path.join(work_dir,"edu_regex_publish.txt"), 
                                                       n_jobs=8,
                                                       verbose = True)
jl.dump([predict_domain, predict_dict], "./data/predict_edu.jl.z")

batch index 0	sample index 0	hit url 1	/api/client/verify_channel
batch index 0	sample index 1	hit url 1	/JavaScript/jquery\.js
batch index 1	sample index 0	hit url 4	/WCQEDU/u/cms
batch index 2	sample index 0	hit url 8	/xl/360wm/lfc/pc
batch index 1	sample index 1	hit url 2	/jifen/shenzhen/bd/pc
batch index 0	sample index 2	hit url 6	/wap/toutiao
batch index 1	sample index 2	hit url 1	/static/mobile/css/reset\.css
batch index 3	sample index 0	hit url 5	/xl/gd/gz
batch index 2	sample index 1	hit url 6	/xl/360wm/lfc/pc/\d{1}
batch index 0	sample index 3	hit url 17	/content/js
batch index 3	sample index 1	hit url 1	/flash/focus\.swf
batch index 1	sample index 3	hit url 22	/newpower/baidu/search
batch index 2	sample index 2	hit url 8	/api/course
batch index 4	sample index 0	hit url 1	/kns/brief/brief\.aspx
batch index 0	sample index 4	hit url 7	/tuiguang/projects/jijin
batch index 3	sample index 2	hit url 3	/newpower/baidu/search/psy
batch index 5	sample index 0	hit url 4	/jifen/shenzhen


['./data/predict_edu.jl.z']

In [163]:
cmt_edu = []
predict_domain, predict_dict =  jl.load("./data/predict_edu.jl.z")
predict_domain = set(predict_domain) - set(cmt_die) - set([_ for k, __ in fp_map.iteritems() for _ in __])
new_detect = predict_domain - set(ac_edu_domain)

print "total detection:%d" %len(predict_domain)
print "predict porn(trainset/independent):\t%d/%d" %(len(predict_domain & set(ac_edu_domain)), len(predict_domain & set(cmt_edu)))
print "new detect:\t%d" %len(new_detect)

dump_urls(list(new_detect), "./data/predict_edu.txt")

total detection:256
predict porn(trainset/independent):	125/0
new detect:	131
